# Capstone Project - Week 5
### Applied Data Science Specialization
<hr>

## A Dashboard of the Popular Establishments In South Africa’s Three Capital Cities


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">
<font size = 3>
    
1.  <a href="#intro">Introduction/Business Problem</a>
    
2.  <a href="#data">Data</a>
    
3.  <a href="#methodology">Methodology</a> 
    
4.  <a href="#analysis">Analysis</a> 

5.  <a href="#results">Results and Discussion</a> 
    
6.  <a href="#conclusion">Conclusion</a>     

</font>
</div>
<hr>

<a id='intro'></a>
## 1. Introduction/Business Problem

In efforts to continuously encourage people to explore South Africa, my project will develop a dashboard that will display popular establishments located in the country’s three capital cities: Bloemfontein, Cape Town, and Pretoria.

This dashboard can be useful to the following stakeholders:
1. Locals
2. Tourists
3. The South African Department of Tourism

Both locals and tourists can use this dashboard as a source of content to plan their bucket-list experiences.
The Department of Tourism can use the dashboard to know which establishments should be featured in the Sho’t Left and Discover South Africa campaigns that they run.

The following details of the popular establishments will be indicated: the establishment’s name, the type of establishment, and the street address.

Below is an example of the expected dashboard layout, colours are used for example purposes:
<img src="../Week4/dashboard_layout.png" width ="800" height=800 >
<hr>

<a id='data'></a>
## 2. Data

This project will use the Foursquare location data as it contains names of the popular establishments, including, but not limited to, the category, address and geographical coordinates (longitude and latitude), city, state and country.

Using my existing Foursquare developer account credentials, of which I have already obtained the access token, I will be accessing each of the three South African capital city's data.
<hr>

<a id='methodology'></a>
## 3. Methodology
This project will implement the following three steps:

**A. Data collection**

Data indicating popular establishments in the three capital cities (Pretoria, Bloemfontein, and Cape Town) of South Africa were downloaded from the Foursquare location data (as explained above in the <a href="#data">Data</a> section).

**B. Pre-processing**

To ensure that quality data is used in the downstream analysis, all data from the three capital cities will be subjected to this cleaning processes will entails the following two steps:

   1. check and remove records with missing establishment name.
   2. check and remove records with missing category.
   3. replace missing addresses.
   4. filter data by selecting columns of interests only.
    
**C. Analysis**

This step includes the following:
    
   1. visualize the popular establishments in each capital city.
   2. find the total number of unique categories in each capital city.
   3. list the unique categories and their corresponding total number of popular establishments.
   4. create a dashboard to visualize the popular establishments in each capital city.

<hr>

## Data collection

Data indicating popular establishments in the three capital cities (Pretoria, Bloemfontein, and Cape Town) of South Africa were downloaded from the Foursquare location data (as explained above in the <a href="#data">Data</a> section).


In [1]:
#installations and imports
print('----- Start Installing and Importing -----\n')

!pip install bs4 #BeautifulSoup version 4
from bs4 import BeautifulSoup #module for web scrapping.
import requests  #module for downloading a web page
import pandas as pd
import numpy as np
import re #module to work with regular expressions

!pip install geopy
from geopy.geocoders import Nominatim #module to convert an address into latitude and longitude values

#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import requests

! pip install folium==0.5.0
import folium #map rendering library

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('\n----- Libraries Imported -----')

----- Start Installing and Importing -----


----- Libraries Imported -----


In [2]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [3]:
#function to download and filter the data for each capital city
def get_selected_province_data(address, agent_name):

    geolocator = Nominatim(user_agent=agent_name)
    location = geolocator.geocode(address)
    print("coordinates: " + str(location.latitude) + " " + str(location.longitude))
    
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, location.latitude, location.longitude, VERSION, radius, LIMIT)
    print(url)
    
    results = requests.get(url).json()
    print('There are {} popular places.'.format(len(results['response']['groups'][0]['items'])))
          
    items = results['response']['groups'][0]['items']
          
    dataframe = json_normalize(items) # flatten JSON

    #filter columns
    filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
    dataframe_filtered = dataframe.loc[:, filtered_columns]

    #filter the category for each row
    dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    #clean columns
    dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]
   
    return [location.latitude, location.longitude, dataframe_filtered]
    

In [4]:
CLIENT_ID = 'C1EWMNOXWGFH0EQF2IR0TMK1PQTL2A5BDTQBFYUQTAQGO0HT' 
CLIENT_SECRET = 'BRN4LIWSKQMMHEHIDCHH4RPAMEF5FN3VIHHAHS040NYLMFBM' 
ACCESS_TOKEN = 'C2UXJVKGRHTDY55HR0VIBETKL2JBGBUTIXA3XQKMOGLLGU11' 
VERSION = '20180604'
LIMIT = 30
radius = 1000
print('Credentails: ' + '\nCLIENT_ID: ' + CLIENT_ID + '\nCLIENT_SECRET:' + CLIENT_SECRET)

Credentails: 
CLIENT_ID: C1EWMNOXWGFH0EQF2IR0TMK1PQTL2A5BDTQBFYUQTAQGO0HT
CLIENT_SECRET:BRN4LIWSKQMMHEHIDCHH4RPAMEF5FN3VIHHAHS040NYLMFBM


In [5]:
#pretoria
pta_lat, pta_lng, pta_data = get_selected_province_data("Pretoria, SA", "pta_agent")
print("Pretoria shape (rows, cols):", pta_data.shape)
pta_data

coordinates: -25.7459374 28.1879444
https://api.foursquare.com/v2/venues/explore?client_id=C1EWMNOXWGFH0EQF2IR0TMK1PQTL2A5BDTQBFYUQTAQGO0HT&client_secret=BRN4LIWSKQMMHEHIDCHH4RPAMEF5FN3VIHHAHS040NYLMFBM&ll=-25.7459374,28.1879444&v=20180604&radius=1000&limit=30
There are 25 popular places.
Pretoria shape (rows, cols): (25, 15)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Café Riche,Café,2 Church Square,Church & Paul Kruger Streets,-25.746579,28.187304,"[{'label': 'display', 'lat': -25.7465792155894...",96,0001,ZA,IPitoli,IGauteng,iNingizimu Afrika,[2 Church Square (Church & Paul Kruger Streets...,4b9a2331f964a520c4a035e3
1,State Theatre,Performing Arts Venue,320 Pretorius St.,at Prinsloo St.,-25.746167,28.195084,"[{'label': 'display', 'lat': -25.7461668516173...",716,0002,ZA,IPitoli,IGauteng,iNingizimu Afrika,"[320 Pretorius St. (at Prinsloo St.), IPitoli,...",4b97d9b7f964a520f21935e3
2,Kruger House Museum,History Museum,60 Church Street,NaN,-25.746919,28.181180,"[{'label': 'display', 'lat': -25.7469190862281...",687,NaN,ZA,IPitoli,IGauteng,iNingizimu Afrika,"[60 Church Street, IPitoli, iNingizimu Afrika]",4ec791ac4690116a8c2fbd55
3,Pretoria Zoo,Exhibit,NaN,NaN,-25.740416,28.188903,"[{'label': 'display', 'lat': -25.7404157186895...",622,NaN,ZA,IPitoli,IGauteng,iNingizimu Afrika,"[IPitoli, iNingizimu Afrika]",523c5c8411d22856753c892a
4,TriBeCa Coffee Shop,Coffee Shop,220 Vermuelen Street,NaN,-25.744835,28.188936,"[{'label': 'display', 'lat': -25.7448346731969...",157,NaN,ZA,IPitoli,IGauteng,iNingizimu Afrika,"[220 Vermuelen Street, IPitoli, iNingizimu Afr...",4cca8854c4d06dcbef5c6303
5,Church Square,Plaza,NaN,NaN,-25.746366,28.188006,"[{'label': 'display', 'lat': -25.7463656833679...",48,NaN,ZA,NaN,NaN,iNingizimu Afrika,[iNingizimu Afrika],4c177f57c2dfc928ff12a869
6,Sammy Marks Square,Shopping Mall,"314 Church St.,Pretoria Central",at Lilian Ngoyi St.,-25.745665,28.193570,"[{'label': 'display', 'lat': -25.7456651286787...",564,0002,ZA,IPitoli,IGauteng,iNingizimu Afrika,"[314 Church St.,Pretoria Central (at Lilian Ng...",4d4ce5acb1fa236a70a48829
7,Wimpy,Burger Joint,"Opera Plaza Building, Pretorius Street",NaN,-25.748014,28.189257,"[{'label': 'display', 'lat': -25.7480138127123...",265,0002,ZA,IPitoli,IGauteng,iNingizimu Afrika,"[Opera Plaza Building, Pretorius Street, IPito...",4d8999e17139b1f72ecbb8d4
8,Protea Hotel Capital,Hotel,390 Van Der Walt,NaN,-25.751563,28.193626,"[{'label': 'display', 'lat': -25.751563, 'lng'...",846,0001,ZA,Pretoria,ZA-GT,iNingizimu Afrika,"[390 Van Der Walt, Pretoria, 0001, iNingizimu ...",4bc7949393bdeee1cd8837ae
9,Nando's,Portuguese Restaurant,Lilian Ngoyi St,NaN,-25.748352,28.193476,"[{'label': 'display', 'lat': -25.748352, 'lng'...",616,0002,ZA,IPitoli,IGauteng,iNingizimu Afrika,"[Lilian Ngoyi St, IPitoli, 0002, iNingizimu Af...",4ea888f77bebb7d9db47a001


In [6]:
#bloemfontein
bfn_lat, bfn_lng, bfn_data = get_selected_province_data("Bloemfontein, SA", "bfn_agent")
print("Bloemfontein shape (rows, cols):", bfn_data.shape)
bfn_data

coordinates: -29.116395 26.215496
https://api.foursquare.com/v2/venues/explore?client_id=C1EWMNOXWGFH0EQF2IR0TMK1PQTL2A5BDTQBFYUQTAQGO0HT&client_secret=BRN4LIWSKQMMHEHIDCHH4RPAMEF5FN3VIHHAHS040NYLMFBM&ll=-29.116395,26.215496&v=20180604&radius=1000&limit=30
There are 12 popular places.
Bloemfontein shape (rows, cols): (12, 16)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,Free State Stadium (Vodacom Park),Football Stadium,Att Horak Ave,-29.117375,26.208830,"[{'label': 'display', 'lat': -29.1173746809503...",657,ZA,Manguang/Bloemfontein,South Africa,iNingizimu Afrika,"[Att Horak Ave, Manguang/Bloemfontein, iNingiz...",NaN,NaN,NaN,4c04a0d73022952100cade81
1,BON Hotel Bloemfontein Central,Hotel,"Bloem Plaza, East Burger Street",-29.116834,26.223237,"[{'label': 'display', 'lat': -29.1168337478054...",754,ZA,IBloemfontein,Orange Free State,iNingizimu Afrika,"[Bloem Plaza, East Burger Street, IBloemfontei...",NaN,NaN,NaN,4e3ff4cbaeb73139a198f021
2,Nu Metro,Multiplex,"105 Henry Street, Westdene",-29.115886,26.209062,"[{'label': 'display', 'lat': -29.1158862438600...",628,ZA,IBloemfontein,Orange Free State,iNingizimu Afrika,"[105 Henry Street, Westdene (cnr 1st Avenue), ...",cnr 1st Avenue,9301,NaN,4c15cec682a3c9b68c67fff8
3,Braza,Portuguese Restaurant,NaN,-29.115152,26.209930,"[{'label': 'display', 'lat': -29.1151524669608...",558,ZA,NaN,NaN,iNingizimu Afrika,[iNingizimu Afrika],NaN,NaN,NaN,4c1fa62b63750f479375bb67
4,Ocean Basket,Seafood Restaurant,NaN,-29.115183,26.210192,"[{'label': 'display', 'lat': -29.1151831602441...",533,ZA,IBloemfontein,Orange Free State,iNingizimu Afrika,"[IBloemfontein, iNingizimu Afrika]",NaN,NaN,NaN,4cac9f4697c8a1cd806ca2a5
5,Primi Piatti,Pizza Place,NaN,-29.114553,26.210523,"[{'label': 'display', 'lat': -29.1145528476109...",525,ZA,IBloemfontein,Orange Free State,iNingizimu Afrika,"[IBloemfontein, iNingizimu Afrika]",NaN,NaN,NaN,4df48845a809141629a655d3
6,KFC,Fast Food Restaurant,Zastron-Markgraaff,-29.112700,26.215300,"[{'label': 'display', 'lat': -29.1127, 'lng': ...",411,ZA,IBloemfontein,Orange Free State,iNingizimu Afrika,"[Zastron-Markgraaff, IBloemfontein, 9301, iNin...",NaN,9301,NaN,4d5c0574fb186dcbcac1fa9a
7,McDonald's,Fast Food Restaurant,NaN,-29.115414,26.209942,"[{'label': 'display', 'lat': -29.1154141370091...",551,ZA,NaN,NaN,iNingizimu Afrika,[iNingizimu Afrika],NaN,NaN,NaN,4d5a5b9a1d6cf04d1adc40fe
8,Wimpy,Restaurant,"Shop LG04 Loch Logan Waterfront, Henry Street,...",-29.115267,26.211169,"[{'label': 'display', 'lat': -29.1152674, 'lng...",439,ZA,IBloemfontein,Orange Free State,iNingizimu Afrika,"[Shop LG04 Loch Logan Waterfront, Henry Street...",NaN,9301,NaN,4d50dd629d4937047b9dc639
9,Falcon Lake Spur,Breakfast Spot,"107 Henry St, Shop No. UG 38, Loch Logan Water...",-29.115209,26.209785,"[{'label': 'display', 'lat': -29.1152087523965...",570,ZA,IBloemfontein,Orange Free State,iNingizimu Afrika,"[107 Henry St, Shop No. UG 38, Loch Logan Wate...",NaN,9301,NaN,4d008ac8ba1da1cd163e8928


In [7]:
#cape town
cpt_lat, cpt_lng, cpt_data = get_selected_province_data("Cape Town, SA", "cpt_agent")
print("Cape Town shape (rows, cols):", cpt_data.shape)
cpt_data

coordinates: -33.928992 18.417396
https://api.foursquare.com/v2/venues/explore?client_id=C1EWMNOXWGFH0EQF2IR0TMK1PQTL2A5BDTQBFYUQTAQGO0HT&client_secret=BRN4LIWSKQMMHEHIDCHH4RPAMEF5FN3VIHHAHS040NYLMFBM&ll=-33.928992,18.417396&v=20180604&radius=1000&limit=30
There are 30 popular places.
Cape Town shape (rows, cols): (30, 16)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,South African Jewish Museum,Museum,88 Hatfield St.,-33.930046,18.416916,"[{'label': 'display', 'lat': -33.9300455262218...",125,8001,ZA,iKapa,Western Cape,iNingizimu Afrika,"[88 Hatfield St., iKapa, 8001, iNingizimu Afrika]",NaN,NaN,4b657e01f964a52046f22ae3
1,The Company's Garden,Park,Queen Victoria St,-33.926615,18.417031,"[{'label': 'display', 'lat': -33.9266150853607...",266,8001,ZA,iKapa,Western Cape,iNingizimu Afrika,"[Queen Victoria St, iKapa, 8001, iNingizimu Af...",NaN,NaN,4b615fb6f964a52064112ae3
2,Maria's,Greek Restaurant,Dunkley Square,-33.931442,18.416777,"[{'label': 'display', 'lat': -33.9314418829224...",278,NaN,ZA,iKapa,Western Cape,iNingizimu Afrika,"[Dunkley Square, iKapa, iNingizimu Afrika]",NaN,NaN,4d6cd9cf423d5481d8b3e5b1
3,The Book Lounge,Bookstore,71 Roeland St,-33.929306,18.421688,"[{'label': 'display', 'lat': -33.9293061422990...",398,NaN,ZA,iKapa,Western Cape,iNingizimu Afrika,"[71 Roeland St (Buitekant St), iKapa, iNingizi...",Buitekant St,NaN,4b9e4a4cf964a5209dd736e3
4,Royale Eatery,Burger Joint,273 Long St.,-33.926425,18.414894,"[{'label': 'display', 'lat': -33.9264249952930...",367,8000,ZA,iKapa,Western Cape,iNingizimu Afrika,"[273 Long St., iKapa, 8000, iNingizimu Afrika]",NaN,NaN,4b5b0a2ef964a5204ee028e3
5,The Yard,Burger Joint,6 Roodehek St,-33.932358,18.419240,"[{'label': 'display', 'lat': -33.9323576304062...",411,8001,ZA,Gardens,Western Cape,iNingizimu Afrika,"[6 Roodehek St (Buitenkant St), Gardens, 8001,...",Buitenkant St,NaN,5049decbd63eaea07a7701d2
6,Kloof Street House,African Restaurant,30 Kloof St,-33.928305,18.412194,"[{'label': 'display', 'lat': -33.9283045015845...",486,8001,ZA,Cape Town,Gardens,iNingizimu Afrika,"[30 Kloof St, Cape Town, 8001, iNingizimu Afrika]",NaN,NaN,54a88b5c498e72e47a96233f
7,Lola's Café,Café,261 Long St,-33.926067,18.415059,"[{'label': 'display', 'lat': -33.9260672817769...",390,8001,ZA,iKapa,Western Cape,iNingizimu Afrika,"[261 Long St (Buiten St), iKapa, 8001, iNingiz...",Buiten St,NaN,4b586bc7f964a520f85628e3
8,BEERHOUSE,Gastropub,223 Long Street,-33.925399,18.416083,"[{'label': 'display', 'lat': -33.925399, 'lng'...",417,8001,ZA,iKapa,Western Cape,iNingizimu Afrika,"[223 Long Street, iKapa, 8001, iNingizimu Afrika]",NaN,Cape Town CBD - Foreshore,50c897e7e4b0cd965f396404
9,The Cousins Trattoria,Italian Restaurant,3b Barrack St,-33.926677,18.421203,"[{'label': 'display', 'lat': -33.9266772354289...",435,8001,ZA,Cape Town,Western Province,iNingizimu Afrika,"[3b Barrack St, Cape Town, 8001, iNingizimu Af...",NaN,NaN,51379f99e4b04f7260ce8840


## Pre-processing

To ensure that quality data is used in the downstream analysis, all data from the three capital cities will be subjected to this cleaning processes will entails the following two steps:

   1. check and remove records with missing establishment name.
   2. check and remove records with missing category.
   3. replace missing addresses.
   4. filter data by selecting columns of interests only.


In [8]:
def remove_and_replace(df, capital_city_name):
   
    #1. check and remove records with missing establishment name
    missing_names = df[df['name'].isnull()].index.tolist() #getting the index of the row with a missing name
    if (len(missing_names) > 0): #drop the rows
        print("Remove the following " + str(len(missing_names)) + " rows: " + str(missing_names))
        df.drop(df.index[missing_names], inplace=True)      
    else:
        print("No missing names")
     
    #2. check and remove records with missing category
    missing_categories = df[df['categories'].isnull()].index.tolist() #getting the index of the row with a missing category
    if (len(missing_categories) > 0): #drop the rows
        print("Remove the following " + str(len(missing_categories)) + " rows: " + str(missing_categories))
        df.drop(df.index[missing_categories], inplace=True)
    else:
        print("No missing categories")
     
    print('\nThere are {} popular places remaining.'.format(df.shape[0]))
    
    #3. replace missing addresses
    df['address'] = df['address'].replace(np.nan, '---', regex=True)
    print("\nEmpty addresses replaced with '---'")
    
    return df[['name', 'categories', 'address', 'lat', 'lng']] #4. return filtered df
               

In [9]:
#pretoria
pta_filtered = remove_and_replace(pta_data, "Pretoria") 
print("\nPretoria filtered shape (rows, cols):", pta_filtered.shape)
pta_filtered

No missing names
No missing categories

There are 25 popular places remaining.

Empty addresses replaced with '---'

Pretoria filtered shape (rows, cols): (25, 5)


,name,categories,address,lat,lng
0,Café Riche,Café,2 Church Square,-25.746579,28.187304
1,State Theatre,Performing Arts Venue,320 Pretorius St.,-25.746167,28.195084
2,Kruger House Museum,History Museum,60 Church Street,-25.746919,28.181180
3,Pretoria Zoo,Exhibit,---,-25.740416,28.188903
4,TriBeCa Coffee Shop,Coffee Shop,220 Vermuelen Street,-25.744835,28.188936
5,Church Square,Plaza,---,-25.746366,28.188006
6,Sammy Marks Square,Shopping Mall,"314 Church St.,Pretoria Central",-25.745665,28.193570
7,Wimpy,Burger Joint,"Opera Plaza Building, Pretorius Street",-25.748014,28.189257
8,Protea Hotel Capital,Hotel,390 Van Der Walt,-25.751563,28.193626
9,Nando's,Portuguese Restaurant,Lilian Ngoyi St,-25.748352,28.193476


In [10]:
#bloemfontein
bfn_filtered = remove_and_replace(bfn_data, "Bloemfontein") 
print("\nBloemfontein filtered shape (rows, cols):", bfn_filtered.shape)
bfn_filtered

No missing names
No missing categories

There are 12 popular places remaining.

Empty addresses replaced with '---'

Bloemfontein filtered shape (rows, cols): (12, 5)


,name,categories,address,lat,lng
0,Free State Stadium (Vodacom Park),Football Stadium,Att Horak Ave,-29.117375,26.208830
1,BON Hotel Bloemfontein Central,Hotel,"Bloem Plaza, East Burger Street",-29.116834,26.223237
2,Nu Metro,Multiplex,"105 Henry Street, Westdene",-29.115886,26.209062
3,Braza,Portuguese Restaurant,---,-29.115152,26.209930
4,Ocean Basket,Seafood Restaurant,---,-29.115183,26.210192
5,Primi Piatti,Pizza Place,---,-29.114553,26.210523
6,KFC,Fast Food Restaurant,Zastron-Markgraaff,-29.112700,26.215300
7,McDonald's,Fast Food Restaurant,---,-29.115414,26.209942
8,Wimpy,Restaurant,"Shop LG04 Loch Logan Waterfront, Henry Street,...",-29.115267,26.211169
9,Falcon Lake Spur,Breakfast Spot,"107 Henry St, Shop No. UG 38, Loch Logan Water...",-29.115209,26.209785


In [11]:
#cape town
cpt_filtered = remove_and_replace(cpt_data, "Cape Town") 
print("\nCape Town filtered shape (rows, cols):", cpt_filtered.shape)
cpt_filtered

No missing names
No missing categories

There are 30 popular places remaining.

Empty addresses replaced with '---'

Cape Town filtered shape (rows, cols): (30, 5)


,name,categories,address,lat,lng
0,South African Jewish Museum,Museum,88 Hatfield St.,-33.930046,18.416916
1,The Company's Garden,Park,Queen Victoria St,-33.926615,18.417031
2,Maria's,Greek Restaurant,Dunkley Square,-33.931442,18.416777
3,The Book Lounge,Bookstore,71 Roeland St,-33.929306,18.421688
4,Royale Eatery,Burger Joint,273 Long St.,-33.926425,18.414894
5,The Yard,Burger Joint,6 Roodehek St,-33.932358,18.419240
6,Kloof Street House,African Restaurant,30 Kloof St,-33.928305,18.412194
7,Lola's Café,Café,261 Long St,-33.926067,18.415059
8,BEERHOUSE,Gastropub,223 Long Street,-33.925399,18.416083
9,The Cousins Trattoria,Italian Restaurant,3b Barrack St,-33.926677,18.421203


<a id='analysis'></a>
## 4. Analysis
This will implement the following two steps:

 4.1. <a href="#map">Visualize Popular Establishments</a>
    
 4.2.  <a href="#dashboard">Create Dashboard</a>
<hr>

<a id='map'></a>
## 4.1. Visualize Popular Establishments

Use the Folium library to visualize the popular establishments in each capital city.

Add the popular establishments to the map as blue markers.

Clicking on each marker should display the following details of each popular establishment: 
- the establishment’s name, written in bold.
- the type of establishment, written inside brackets.
- the street address, written in italics.
<hr>

In [12]:
def generate_map(df_lat, df_lng, df_data, html_filename):
    venues_map = folium.Map(location=[df_lat, df_lng], zoom_start=15) #generate map 
 
    #add popular establishments to the map as blue markers
    for lat, lng, nme, cat, addr in zip(df_data.lat, df_data.lng, df_data.name, df_data.categories, df_data.address):
        html= folium.Html('<p style="text-align:center; font-size: 12px;"><b>' + nme + '</b><br>(' + cat + ')<br><i>' + str(addr) + '</i></p>', script=True)
        iframe = folium.IFrame(html=html, width=150, height=70)
        folium.Marker([lat, lng], popup=folium.Popup(iframe)).add_to(venues_map)

    venues_map.save(html_filename) #save map as .html file to be used in the dashboard application layout

In [13]:
#pretoria
generate_map(pta_lat, pta_lng, pta_filtered, "PTA_Establishments.html")

#bloemfontein
generate_map(bfn_lat, bfn_lng, bfn_filtered, "BFN_Establishments.html")

#cape town
generate_map(cpt_lat, cpt_lng, cpt_filtered, "CPT_Establishments.html")

print("Maps Generated and Saved")

Maps Generated and Saved


<a id='dashboard'></a>
## 4.2. Create Dashboard

As per the dashboard template (displayed in the <a href='#intro'>Introduction/Business Problem</a> section above), the additional information section should include the following:

1. the total number of unique categories in each capital city.
2. table that lists the unique categories and their corresponding total number of popular establishments.
<hr>

In [14]:
#imports
print('----- Start Installing and Importing -----\n')

!pip install jupyter_dash
from jupyter_dash import JupyterDash

!pip install dash
import dash
import dash_html_components as html

print('\n----- Libraries Imported -----')

----- Start Installing and Importing -----


----- Libraries Imported -----


In [18]:
#create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

In [16]:
#function to explore the establishments in each capital city
def generate_table(dataframe): #2. create a table that lists the unique categories and their corresponding total number of popular establishments
    categories_df = (dataframe.groupby(['categories']).size()).reset_index()

    return html.Table(#header
                      [html.Tr([html.Th('Categories'), html.Th('Number of Establishments')])] +
                      #body
                      [html.Tr([html.Td(categories_df.loc[i]["categories"]), html.Td(categories_df.loc[i][0])])  
                       for i in range(0, len(categories_df))],
                      style={'border':'0.2px solid black', 'textAlign':'center'}
                     )

In [19]:
#application layout
app.layout = html.Div(style={'backgroundColor':'#060b6c', 'height':'1850px'},
                      children=[ #title and sub-title div 
                                 html.Div([html.H1('Explore South Africa', style={'textAlign':'center', 'color':'#fcfcfc', 'font-size':40, 'margin-top':'2px'}), #dashboard title
                                           html.H3('Fun Awaits You!', style={'textAlign':'center', 'color':'#fcfcfc', 'font-size':20, 'margin-top':'-25px'}), #dashboard sub-title
                                          ],
                                          style={'backgroundColor':'#0978ca', 'width':'96%', 'height':'80px', 'position':'absolute', 'margin':'10px'}),
                                 
                                 #pretoria div
                                 html.Br(),
                                 html.Div(children=[html.H4('Pretoria, Gauteng Province', style={'textAlign':'center', 'color':'black', 'font-size':18, 'margin':'2px'}), #province 1 title
                                                    html.Div(children=[html.Iframe(id='PTAmap', srcDoc=open('PTA_Establishments.html', 'r').read(), width='70%', height='486', style={'margin-left':'2px'}), #map div

                                                                       #additional information div
                                                                       html.Div([html.H5('Additional Information', style={'textAlign':'center', 'color':'black', 'margin-top':'5px', 'font-size':15}),
                                                                                 html.P('Number of Popular Establishments: ' + str(pta_filtered.shape[0]), style={'margin-left':'5px'}),
                                                                                 generate_table(pta_filtered),
                                                                                ],
                                                                                style={'backgroundColor':'#bae8ac', 'width':'29%', 'height':'490px', 'position':'absolute', 'margin-left':'670px', 'font-size':14}),
                                                                      ],
                                                            style={'display':'flex'}),
                                                    ],
                                          style={'backgroundColor':'#ffffff', 'width':'96%', 'height':'520px', 'position':'absolute', 'margin':'80px 10px'},),
                                 
                                 #bloemfontein div
                                 html.Br(),
                                 html.Div(children=[html.H4('Bloemfontein, Free State Province', style={'textAlign':'center', 'color':'black', 'font-size':18, 'margin':'2px'}), #province 2 title
                                                    html.Div(children=[html.Iframe(id='BLFmap', srcDoc=open('BFN_Establishments.html', 'r').read(), width='70%', height='465', style={'margin-left':'2px'}), #map div

                                                                       #additional information div
                                                                       html.Div([html.H5('Additional Information', style={'textAlign':'center', 'color':'black', 'margin-top':'5px', 'font-size':15}),
                                                                                 html.P('Number of Popular Establishments: ' + str(bfn_filtered.shape[0]), style={'margin-left':'5px'}),
                                                                                 generate_table(bfn_filtered),
                                                                                ],
                                                                                style={'backgroundColor':'#bae8ac', 'width':'29%', 'height':'470px', 'position':'absolute', 'margin-left':'670px', 'font-size':14}),
                                                                      ],
                                                            style={'display':'flex'}),
                                                    ],
                                          style={'backgroundColor':'#ffffff', 'width':'96%', 'height':'500px', 'position':'absolute', 'margin':'590px 10px'},),
                                 
                                 #cape town div
                                 html.Br(),
                                 html.Div(children=[html.H4('Cape Town, Western Cape Province', style={'textAlign':'center', 'color':'black', 'font-size':18, 'margin':'2px'}), #province 3 title
                                                    html.Div(children=[html.Iframe(id='CPTmap', srcDoc=open('CPT_Establishments.html', 'r').read(), width='70%', height='655', style={'margin-left':'2px'}), #map div

                                                                       #additional information div
                                                                       html.Div([html.H5('Additional Information', style={'textAlign':'center', 'color':'black', 'margin-top':'5px', 'font-size':15}),
                                                                                 html.P('Number of Popular Establishments: ' + str(cpt_filtered.shape[0]), style={'margin-left':'5px'}),
                                                                                 generate_table(cpt_filtered),
                                                                                ],
                                                                                style={'backgroundColor':'#bae8ac', 'width':'29%', 'height':'660px', 'position':'absolute', 'margin-left':'670px', 'font-size':14}),
                                                                      ],
                                                            style={'display':'flex'}),
                                                    ],
                                          style={'backgroundColor':'#ffffff', 'width':'96%', 'height':'695px', 'position':'absolute', 'margin':'1080px 10px'},), 
                            ]
                     )  

if __name__ == '__main__':
    app.run_server(mode="inline", host="localhost", debug=True, dev_tools_ui=False, dev_tools_props_check=False)
        

### If the dashboard maps are not visible on Github, please refer to the corresponding Report (Capstone_Project_Week5_Report.pdf) or the Presentation (Capstone_Project_Week5_Presentation.pdf) on github.

<hr>

<a id='results'></a>
## 5. Results and Discussion

This analysis shows that there are numerous popular establishments in each of the three capital cities (Pretoria, Bloemfontein, and Cape Town) from South Africa.

Although, the initial downloaded data from the Foursquare location data didn't include any missing names and categories for each popular establishment in each of the three capital cities, there were several establishments that didn't have an address associate to it. For the latter, the string "---" was used as an address.   

In total there were 25, 12, and 30 popular establishments in Pretoria, Bloemfontein, and Cape Town, respectively. With Cape Town having the most popular establishments, this indicates that the capital city has a wider variety of establishments to choose from and mostly, there is something to suit everyone's interests.


Exploring the categories of the popular establishments indicated that 4 of the popular establishments located in Pretoria are Burger Joints. Similarly, 4 of the popular establishments located in Cape Town are Cafes. On the other hand, 2 of the popular establishments in Bloemfontein are Fast Food Restaurants and Portuguese Restaurants.
<hr>

<a id='conclusion'></a>
## 6. Conclusion

This project has managed to create a dashboard that displays the popular establishments in three of South Africa's capital cities: Pretoria, Bloemfontein, and Cape Town.

The layout of the dashboard is aimed to be simple yet informative. Hence, the dashboard offers users an interactive experience by being able to click on the markers on map to find the popular establishment's name, category, and street address. For each capital city, users are also given additional information regarding the total number of popular establishments and the category the establishments.

<hr>